In [1]:
from hdx.hdx_configuration import Configuration 
from hdx.data.dataset import Dataset
import requests
import pandas as pd
import numpy as np
import csv
from zipfile import ZipFile
import os

In [2]:
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

# Population

In [3]:
prediction_data  = pd.read_csv('../data/train/pop/fr/departements-francais.csv', sep=';')
prediction_data.columns = ['dep_num', 'name', 'region', 'capital', 'area', 'total', 'density']
prediction_data = prediction_data.sort_values('dep_num')
prediction_data = prediction_data[:-5]
prediction_data['region'] = prediction_data['region'].replace({'Ile-de-France':'Île-de-France'})
prediction_data

,dep_num,name,region,capital,area,total,density
0,1,Ain,Auvergne-Rhône-Alpes,Bourg-en-Bresse,5762,631877,109.7
1,2,Aisne,Hauts-de-France,Laon,7369,538659,73.1
2,3,Allier,Auvergne-Rhône-Alpes,Moulins,7340,341613,46.5
3,4,Alpes-de-Haute-Provence,Provence-Alpes-Côte d'Azur,Digne,6925,161799,23.4
4,5,Hautes-Alpes,Provence-Alpes-Côte d'Azur,Gap,5549,140916,25.4
...,...,...,...,...,...,...,...
93,93,Seine-Saint-Denis,Île-de-France,Bobigny,236,1592663,6742.9
94,94,Val-de-Marne,Île-de-France,Créteil,245,1372389,5601.6
95,95,Val-d'Oise,Île-de-France,Pontoise,1246,1215390,975.5
19,201,Corse-du-Sud,Corse,Ajaccio,4014,152730,38.0


# Covid

In [4]:
#import requests
#import pandas as pd
#import datetime
#departements = pd.read_csv("data/pop/fr/departements-francais.csv", sep = ";")
#hospi = []
#url = "https://coronavirusapi-france.now.sh/AllLiveData"
#response = requests.get(url).json()
#counter = 0
#for numero in departements.NUMÉRO:
#    nom = str(departements[departements["NUMÉRO"]==numero]["NOM"].values[0])
#    hospi.append((nom, numero, response["allLiveFranceData"][counter]["nouvellesHospitalisations"]))
#    counter+=1
#
#df = pd.DataFrame(hospi, columns =["depname","depnum","newhospi"])
#print(df)
#df.to_csv("covid_daily_data.csv", index = False)

# CAMS

In [5]:
#to_do (Niclas ?)

# Mobility

In [6]:
Configuration.create(hdx_site='prod', user_agent='A_Quick_Example', hdx_read_only=True)
dataset = Dataset.read_from_hdx('movement-range-maps')
resources = dataset.get_resources()
dic = resources[1]
url_mobility = dic['download_url']

file_mobility = "../data/prediction/mvt_range.zip"
download_url(url_mobility, file_mobility)

In [7]:
with ZipFile(file_mobility, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    print('Extracting mv_range file now...')
    mvt_range = zip.namelist()[-1]
    zip.extract(mvt_range)
    print('Done!')

File Name                                             Modified             Size
README.txt                                     2021-04-18 07:44:38          961
movement-range-2021-04-17.txt                  2021-04-18 07:44:50    571941581
Extracting mv_range file now...
Done!


In [8]:
with open(mvt_range) as f:
    reader = csv.reader(f, delimiter="\t")
    d = list(reader)

In [9]:
data_mob = pd.DataFrame(d[1:], columns=d[0])
data_mob = data_mob[data_mob['country']=='FRA']

In [10]:
data_mob = data_mob[data_mob['ds']==list(data_mob.iloc[[-1]]['ds'])[0]][['ds', 'polygon_name', 'all_day_bing_tiles_visited_relative_change', 'all_day_ratio_single_tile_users']]

In [11]:
prediction_data['stay_put'] = 0.0
prediction_data['go_out'] = 0.0

In [12]:
def add_go_out(row):
    region = row['region']
    go_out = data_mob[data_mob['polygon_name']==region]['all_day_bing_tiles_visited_relative_change']
    return float(list(go_out)[0])
     
def add_stay_put(row):   
    region = row['region']
    stay_put = data_mob[data_mob['polygon_name']==region]['all_day_ratio_single_tile_users']
    return float(list(stay_put)[0])

In [13]:
prediction_data['go_out'] = prediction_data.apply(add_go_out, axis=1)
prediction_data['stay_put'] = prediction_data.apply(add_stay_put, axis=1)
prediction_data

,dep_num,name,region,capital,area,total,density,stay_put,go_out
0,1,Ain,Auvergne-Rhône-Alpes,Bourg-en-Bresse,5762,631877,109.7,0.25005,-0.28998
1,2,Aisne,Hauts-de-France,Laon,7369,538659,73.1,0.24126,-0.20317
2,3,Allier,Auvergne-Rhône-Alpes,Moulins,7340,341613,46.5,0.25005,-0.28998
3,4,Alpes-de-Haute-Provence,Provence-Alpes-Côte d'Azur,Digne,6925,161799,23.4,0.29558,-0.33294
4,5,Hautes-Alpes,Provence-Alpes-Côte d'Azur,Gap,5549,140916,25.4,0.29558,-0.33294
...,...,...,...,...,...,...,...,...,...
93,93,Seine-Saint-Denis,Île-de-France,Bobigny,236,1592663,6742.9,0.24841,-0.21480
94,94,Val-de-Marne,Île-de-France,Créteil,245,1372389,5601.6,0.24841,-0.21480
95,95,Val-d'Oise,Île-de-France,Pontoise,1246,1215390,975.5,0.24841,-0.21480
19,201,Corse-du-Sud,Corse,Ajaccio,4014,152730,38.0,0.25426,-0.20294


# Vaccins

In [27]:
url_positive_test = 'https://www.data.gouv.fr/es/datasets/r/59aeab47-c364-462c-9087-ce233b6acbbc'
download_url(url_positive_test, '../data/prediction/live_vaccins.csv')

In [59]:
live_vacc = pd.read_csv('../data/prediction/live_vaccins.csv')
live_vacc['date_debut_semaine'] = pd.to_datetime(vacc_1_reg['date_debut_semaine'])
date_max = vacc_1_reg['date_debut_semaine'].max()

In [31]:
vacc_1 = live_vacc[live_vacc['rang_vaccinal']==1]
vacc_2 = live_vacc[live_vacc['rang_vaccinal']==2]

In [21]:
prediction_data

,dep_num,name,region,capital,area,total,density,stay_put,go_out
0,1,Ain,Auvergne-Rhône-Alpes,Bourg-en-Bresse,5762,631877,109.7,0.25005,-0.28998
1,2,Aisne,Hauts-de-France,Laon,7369,538659,73.1,0.24126,-0.20317
2,3,Allier,Auvergne-Rhône-Alpes,Moulins,7340,341613,46.5,0.25005,-0.28998
3,4,Alpes-de-Haute-Provence,Provence-Alpes-Côte d'Azur,Digne,6925,161799,23.4,0.29558,-0.33294
4,5,Hautes-Alpes,Provence-Alpes-Côte d'Azur,Gap,5549,140916,25.4,0.29558,-0.33294
...,...,...,...,...,...,...,...,...,...
93,93,Seine-Saint-Denis,Île-de-France,Bobigny,236,1592663,6742.9,0.24841,-0.21480
94,94,Val-de-Marne,Île-de-France,Créteil,245,1372389,5601.6,0.24841,-0.21480
95,95,Val-d'Oise,Île-de-France,Pontoise,1246,1215390,975.5,0.24841,-0.21480
19,201,Corse-du-Sud,Corse,Ajaccio,4014,152730,38.0,0.25426,-0.20294


In [61]:
vacc_1_reg = vacc_1[vacc_1['code_region']==4]
vacc_1_reg
#vacc_1_reg[vacc_1_reg['date_debut_semaine']==date_max]['nb']

,code_region,region,departement,rang_vaccinal,date_debut_semaine,nb
3333,4,REU,974,1,2021-01-18,191
3334,4,REU,974,1,2021-01-25,1478
3336,4,REU,974,1,2021-02-01,2242
3338,4,REU,974,1,2021-02-08,2615
3340,4,REU,974,1,2021-02-15,2097
3342,4,REU,974,1,2021-02-22,1761
3344,4,REU,974,1,2021-03-01,2200
3346,4,REU,974,1,2021-03-08,2317
3348,4,REU,974,1,2021-03-15,2610
3350,4,REU,974,1,2021-03-22,3093


In [62]:
def live_vacc_1(row):
    dep = row['dep_num']
    vacc_1_reg = vacc_1[vacc_1['code_region']==dep]
    if vacc_1_reg.shape[0]!=0:
        nb_series = vacc_1_reg[vacc_1_reg['date_debut_semaine']==date_max]['nb']
        nb = list(nb_series)[0]
    else : 
        nb=0
    return nb

def live_vacc_2(row):
    dep = row['dep_num']
    vacc_2_reg = vacc_2[vacc_2['code_region']==dep]
    if vacc_2_reg.shape[0]!=0:
        nb_series = vacc_2_reg[vacc_2_reg['date_debut_semaine']==date_max]['nb']
        nb = list(nb_series)[0]
    else : 
        nb=0
    return nb

In [63]:
prediction_data['vacc_1'] = prediction_data.apply(live_vacc_1, axis=1)
prediction_data['vacc_2'] = prediction_data.apply(live_vacc_2, axis=1)
prediction_data

,dep_num,name,region,capital,area,total,density,stay_put,go_out,vacc_1,vacc_2
0,1,Ain,Auvergne-Rhône-Alpes,Bourg-en-Bresse,5762,631877,109.7,0.25005,-0.28998,160,1268
1,2,Aisne,Hauts-de-France,Laon,7369,538659,73.1,0.24126,-0.20317,556,1514
2,3,Allier,Auvergne-Rhône-Alpes,Moulins,7340,341613,46.5,0.25005,-0.28998,406,2473
3,4,Alpes-de-Haute-Provence,Provence-Alpes-Côte d'Azur,Digne,6925,161799,23.4,0.29558,-0.33294,73,4354
4,5,Hautes-Alpes,Provence-Alpes-Côte d'Azur,Gap,5549,140916,25.4,0.29558,-0.33294,0,0
...,...,...,...,...,...,...,...,...,...,...,...
93,93,Seine-Saint-Denis,Île-de-France,Bobigny,236,1592663,6742.9,0.24841,-0.21480,354,1748
94,94,Val-de-Marne,Île-de-France,Créteil,245,1372389,5601.6,0.24841,-0.21480,235,1715
95,95,Val-d'Oise,Île-de-France,Pontoise,1246,1215390,975.5,0.24841,-0.21480,0,0
19,201,Corse-du-Sud,Corse,Ajaccio,4014,152730,38.0,0.25426,-0.20294,0,0


# Positive tests